In [1]:
import mlflow
import os
from dotenv import load_dotenv
from mlflow.tracking import MlflowClient

# Step 2: Set up the MLflow tracking server
load_dotenv()
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))

In [5]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from imblearn.over_sampling import ADASYN

In [3]:
df = pd.read_csv('D:\datascience\campusx\PROJECTS\Second_project\cleaned_data.csv',index_col=0).dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [4]:
mlflow.set_experiment("EXP_5-Xgboost_hpt")

<Experiment: artifact_location='s3://apoorv-mlfow-bucket/650043532633511350', creation_time=1758612848676, experiment_id='650043532633511350', last_update_time=1758612848676, lifecycle_stage='active', name='EXP_5-Xgboost_hpt', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [6]:

# Step 1: Remap class labels
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})
df = df.dropna(subset=['category'])

# Step 2: TF-IDF vectorizer
ngram_range = (1, 3)
max_features = 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 3: ADASYN resampling
adasyn = ADASYN(random_state=42)
X_resampled, y_resampled = adasyn.fit_resample(X, y)

# Step 4: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, best_params=None, study=None):
    with mlflow.start_run():
        # Tags
        mlflow.set_tag("mlflow.runName", f"{model_name}_ADASYN_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Algorithm name
        mlflow.log_param("algo_name", model_name)

        # Log best hyperparameters
        if best_params:
            for param_name, param_value in best_params.items():
                mlflow.log_param(param_name, param_value)

        # Log best Optuna trial info
        if study:
            mlflow.log_param("best_trial_number", study.best_trial.number)
            mlflow.log_metric("best_trial_value", study.best_trial.value)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log metrics
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the trained model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

# Optuna objective for LightGBM
def objective_lightgbm(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = LGBMClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Run Optuna and log best model
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=30)

    best_params = study.best_params
    best_model = LGBMClassifier(
        n_estimators=best_params['n_estimators'],
        learning_rate=best_params['learning_rate'],
        max_depth=best_params['max_depth'],
        random_state=42
    )

    # Log everything to MLflow
    log_mlflow("LightGBM", best_model, X_train, X_test, y_train, y_test, best_params=best_params, study=study)

# Run the experiment
run_optuna_experiment()


[I 2025-09-23 16:40:13,497] A new study created in memory with name: no-name-51f33e91-18db-415b-ba35-5671c800fcd6


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.083448 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 99569
[LightGBM] [Info] Number of data points in the train set: 37398, number of used features: 972
[LightGBM] [Info] Start training from score -1.090702
[LightGBM] [Info] Start training from score -1.086651
[LightGBM] [Info] Start training from score -1.118789
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

c:\Users\apoor\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-09-23 16:40:20,767] Trial 0 finished with value: 0.5036363636363637 and parameters: {'n_estimators': 179, 'learning_rate': 0.00040129338124202725, 'max_depth': 3}. Best is trial 0 with value: 0.5036363636363637.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.075313 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 99569
[LightGBM] [Info] Number of data points in the train set: 37398, number of used features: 972
[LightGBM] [Info] Start training from score -1.090702
[LightGBM] [Info] Start training from score -1.086651
[LightGBM] [Info] Start training from score -1.118789
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

c:\Users\apoor\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-09-23 16:40:32,240] Trial 1 finished with value: 0.5474866310160428 and parameters: {'n_estimators': 280, 'learning_rate': 0.00012563169952741976, 'max_depth': 6}. Best is trial 1 with value: 0.5474866310160428.


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.072375 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 99569
[LightGBM] [Info] Number of data points in the train set: 37398, number of used features: 972
[LightGBM] [Info] Start training from score -1.090702
[LightGBM] [Info] Start training from score -1.086651
[LightGBM] [Info] Start training from score -1.118789
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

c:\Users\apoor\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-09-23 16:44:10,948] Trial 29 finished with value: 0.7190374331550802 and parameters: {'n_estimators': 173, 'learning_rate': 0.056338614671424814, 'max_depth': 3}. Best is trial 21 with value: 0.7993582887700534.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.069433 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 99569
[LightGBM] [Info] Number of data points in the train set: 37398, number of used features: 972
[LightGBM] [Info] Start training from score -1.090702
[LightGBM] [Info] Start training from score -1.086651
[LightGBM] [Info] Start training from score -1.118789
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

c:\Users\apoor\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/09/23 16:44:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/23 16:45:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LightGBM_ADASYN_TFIDF_Trigrams at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/650043532633511350/runs/004beb5431cf46cda3214a7b24846f48
🧪 View experiment at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/650043532633511350
